# View GRAB mismatch closed and open loop, session1 and 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.stats import pearsonr, spearmanr
from analysis_functions import *
from model_functions import *
import matplotlib.patches as patches
import seaborn as sns

In [ ]:
RunTresholdDict = {'B3M1': 145, 'B3M2': 295, 'B3M3': 325, 'B2M4': 110, 'B2M5': 180}

In [ ]:
GRAB_MM_1 = pd.read_csv('GRAB_MMclosed_open_session1.csv', dtype=dtype_dict)
GRAB_MM_2 = pd.read_csv('GRAB_MMclosed_open_session2.csv', dtype=dtype_dict)

In [ ]:
# Chaning event name to halt and making it so that its True when there is a halt
#GRAB_MM_1.loc[:, 'event'] = GRAB_MM_1['event'].replace({False: True, True: False})
GRAB_MM_1.rename(columns = {'event': 'halt'}, inplace = True)
GRAB_MM_2.rename(columns = {'event': 'halt'}, inplace = True)


In [ ]:
#Make sure the index provides inforamtion of seconds since start
GRAB_MM_1.set_index('Seconds', inplace=True)
GRAB_MM_2.set_index('Seconds', inplace=True)
# Theres only photomotry information from 15 seconds due to bleaching
GRAB_MM_1 = GRAB_MM_1.loc[ GRAB_MM_1.index>30]
GRAB_MM_2 = GRAB_MM_2.loc[ GRAB_MM_2.index>30]

In [ ]:
#Check that the same mice are in the datsets
print(GRAB_MM_1.mouseID.unique())
print(GRAB_MM_2.mouseID.unique())


## View session data
- Loop through the mouse names present in one of the dataframes (ideally they should be the same)
- save a variable where each session is saved for the current mouse
- use the view_session_mouse() funciton from analysis_functions.py to plot the Delta F/F 470 fluorescence and movement in X direction with halts in grey, and session blocks marked in colors.
- Edit the function to plot different fluorescence traces, movements, and eyes.
- This is mostly to get an impression of the overall data trends.

Move the function into this file to test it.


In [ ]:
for mouse in GRAB_MM_1.mouseID.unique():
    mousedata = {'session 1':GRAB_MM_1.loc[GRAB_MM_1.mouseID == mouse], 'session 2':GRAB_MM_2.loc[GRAB_MM_2.mouseID == mouse]}
    fig, ax = view_session_mouse(mousedata, mouse)
    fig.savefig(f'Figures/GRAB{mouse}_view_alignment.png', format = 'png', dpi = 300)

In [ ]:
#Filter mice to get unique indexes
mouse_data = {'session 1':{'closedloop': {},'openloop': {} }, 'session 2': {'closedloop': {},'openloop': {}}}

for mouse in GRAB_MM_1.mouseID.unique():
    mouse_data['session 1']['closedloop'][mouse] = filter_data(GRAB_MM_1, filters = [mouse, 'closed_block'])
    mouse_data['session 1']['openloop'][mouse] = filter_data(GRAB_MM_1, filters = [mouse, 'open_block'])
for mouse in GRAB_MM_2.mouseID.unique():
    mouse_data['session 2']['closedloop'][mouse] = filter_data(GRAB_MM_2, filters = [mouse, 'closed_block'])
    mouse_data['session 2']['openloop'][mouse] = filter_data(GRAB_MM_2, filters = [mouse, 'open_block'])

In [ ]:
GRAB_MM_1

In [ ]:
def align_to_event_start(df, trace, event_col, range_around_event):
    
    trace_chunk_list = []
    bsl_trace_chunk_list = []
    run_speed_list = []
    turn_speed_list = []
    event_index_list = []
    
    # Identify the start times for each event
    event_times = df.loc[df[event_col] & ~df[event_col].shift(1, fill_value=False)].index

    # Calculate the time range around each event
    before_0 = range_around_event[0]
    after_0 = range_around_event[1]
    
    # Calculate the target length of each chunk based on the sampling rate
    sampling_rate = 0.001
    target_length = int(((before_0 + after_0) / sampling_rate) + 1)  # Include both ends
    Index= pd.Series(np.linspace(-range_around_event[0], range_around_event[1], target_length)) # common index
    
    for event_time in event_times:
        
        # Determine the time range for each chunk
        start = event_time - before_0
        end = event_time + after_0
        
        # Extract the chunk from the trace column
        chunk = df[trace].loc[start:end]
        runspeed = df['movementX'].loc[start:event_time].mean() #Saving mean run speed up until halt
        turningspeed = df['movementY'].loc[start:event_time].mean() 
        # Normalize the index to start at -before_0
        chunk.index = (chunk.index - chunk.index[0]) - before_0
        # Check if the chunk is shorter than the target length
        if len(chunk) < target_length:
            # Pad the chunk with NaN values at the end to reach the target length
            padding = pd.Series([np.nan] * (target_length - len(chunk)), index=pd.RangeIndex(len(chunk), target_length))
            chunk = pd.concat([chunk, padding])
            chunk.index = Index # Getting the same index as the others
        
        # Baseline the chunk
        baselined_chunk = baseline(chunk)
        
        # Append the chunk and baselined chunk to lists
        trace_chunk_list.append(chunk.values)
        bsl_trace_chunk_list.append(baselined_chunk.values)
        run_speed_list.append(runspeed)
        turn_speed_list.append(turningspeed)
        event_index_list.append(event_time)  # Store the event time for use in final column names
    # Convert lists of arrays to DataFrames
    try:
        trace_chunks = pd.DataFrame(np.column_stack(trace_chunk_list), columns=event_index_list)
        bsl_trace_chunks = pd.DataFrame(np.column_stack(bsl_trace_chunk_list), columns=event_index_list)
        run_speeds = pd.DataFrame(np.column_stack(run_speed_list), columns=event_index_list)
        turn_speeds = pd.DataFrame(np.column_stack(turn_speed_list), columns=event_index_list)
        movement_speeds = pd.concat([run_speeds, turn_speeds])
        
        # Set the index as the common time range index for each chunk
        trace_chunks.index = Index
        bsl_trace_chunks.index = Index
        movement_speeds.index = ['Mean_moveX', 'Mean_moveY'] #set X and Y movement as movement speed index
        
        return trace_chunks, bsl_trace_chunks, movement_speeds
    
    except ValueError:
        if len(event_times) < 1:
            print('could not align to events because there were none, will return nothing')
            
        return 0, 0, 0


In [ ]:
#Aligning open and closed loop block data to halt start times
mouse_aligned = {'session 1':{'closedloop': {},'openloop': {} }, 'session 2': {'closedloop': {},'openloop': {}}}
move_speeds = {'session 1':{'closedloop': {},'openloop': {} }, 'session 2': {'closedloop': {},'openloop': {}}}

for session, session_dict in mouse_data.items():
    for block, mice in session_dict.items():
        for mouse, df in mice.items():
            event_alinged, bsl_event_alinged, run_speeds = align_to_event_start(df, '470_dfF', 'halt',[1,2])
            mouse_aligned[session][block][mouse]  = bsl_event_alinged  #bsl indicates that it is baselined to the last 1 second before halt
            move_speeds[session][block][mouse]  = run_speeds
    

In [ ]:
print(move_speeds['session 1']['closedloop']['B3M7'])
mouse_aligned['session 1']['closedloop']['B3M7']

In [ ]:
len(mouse_aligned['session 1']['closedloop'].keys())

In [ ]:
mouse_aligned_nohalt = {'session 1':{'closedloop': {},'openloop': {} }, 'session 2': {'closedloop': {},'openloop': {}}}
move_speeds_nohalt = {'session 1':{'closedloop': {},'openloop': {} }, 'session 2': {'closedloop': {},'openloop': {}}}
for session, session_dict in mouse_data.items():
    for block, mice in session_dict.items():
        for mouse, df in mice.items():
            event_alinged, bsl_event_alinged, run_speeds = align_to_event_start(df, '470_dfF', 'No_halt',[1,2])
            mouse_aligned_nohalt[session][block][mouse] = bsl_event_alinged  #bsl indicates that it is baselined to the last 1 second before halt
            move_speeds_nohalt[session][block][mouse]  = run_speeds

In [ ]:
mean_mouse_dict_s1 =plot_compare_blocks(mouse_aligned['session 1'], 'halt')
mean_mouse_dict_s2 =plot_compare_blocks(mouse_aligned['session 2'], 'halt')

In [ ]:
mean_mouse_dict_s1_nohalt =plot_compare_blocks(mouse_aligned_nohalt['session 1'], 'No halt')

In [ ]:
mouse_aligned['session 1']['closedloop'].keys()

In [ ]:
fig, axes = plt.subplots(len(mouse_aligned['session 1']['closedloop']),2, figsize = (10,10))
axes = axes.flatten()
i = 0
for (mouse_open, alinged_data_open), (mouse, alinged_data_closed) in zip(mouse_aligned['session 1']['closedloop'].items(), mouse_aligned['session 1']['openloop'].items()):
    ax = axes[i]
    ax.spines[['right', 'top']].set_visible(False)
    #Get no-halt trace
    aligned_nohalt = mouse_aligned_nohalt['session 1']['closedloop'][mouse]

    #Getting mean and standard devaiton
    mouse_data_mean = alinged_data_closed.mean(axis=1)
    mouse_data_std = alinged_data_closed.std(axis=1)
    mouse_nohalt_mean = aligned_nohalt.mean(axis=1)
    mouse_nohalt_std = aligned_nohalt.std(axis=1)

    #Plot main trace 
    ax.plot(mouse_data_mean)
    ax.fill_between(mouse_data_mean.index, mouse_data_mean+mouse_data_std, mouse_data_mean-mouse_data_std, color='cyan', alpha=0.3)
    #plot control trace
    control_trace = ax.plot(mouse_nohalt_mean, color = 'black', label = 'control')
    ax.fill_between(mouse_nohalt_mean.index, mouse_nohalt_mean+mouse_nohalt_std, 
                       mouse_nohalt_mean-mouse_nohalt_std, color='black', alpha=0.1)
    ax.set_title(f'{mouse} closed loop')
    ax.axvline(0, c='grey', ls= '--')
    i+=1
    ax = axes[i]
    ax.spines[['right', 'top']].set_visible(False)
    #Get no-halt trace for open loop
    aligned_nohalt_o = mouse_aligned_nohalt['session 1']['openloop'][mouse]

    #Getting mean and standard devaiton
    mouse_data_mean = alinged_data_open.mean(axis=1)
    mouse_data_std = alinged_data_open.std(axis=1)
    mouse_nohalt_o_mean = aligned_nohalt_o.mean(axis=1)
    mouse_nohalt_o_std = aligned_nohalt_o.std(axis=1)

    #Plot main trace 
    ax.plot(mouse_data_mean)
    ax.fill_between(mouse_data_mean.index, mouse_data_mean+mouse_data_std, mouse_data_mean-mouse_data_std, color='cyan', alpha=0.3)
    #plot control trace
    control_trace = ax.plot(mouse_nohalt_o_mean, color = 'black', label = 'control')
    ax.fill_between(mouse_nohalt_o_mean.index, mouse_nohalt_o_mean+mouse_nohalt_std, 
                       mouse_nohalt_o_mean-mouse_nohalt_o_std, color='black', alpha=0.1)
    ax.set_title(f'{mouse} open loop')
    ax.axvline(0, c='grey', ls= '--')
    i+=1
fig.tight_layout(pad=1.08)

    
    


In [ ]:
sessions = mouse_aligned.keys()

In [ ]:
'Mean_moveX' in move_speeds['session 1']['openloop']['B3M7'].index

In [ ]:
def extract_aligned_vars(aligned_data_dict, aligned_movement_dict):
    # Initialize an empty list to store results
    results = []
    
    for session_number, session_blocks in aligned_data_dict.items():
        for session_block, mice_data in session_blocks.items():
            for mouse_id, item in mice_data.items():
                run_df = aligned_movement_dict[session_number][session_block][mouse_id]
                # Check if the item is a DataFrame
                if not isinstance(item, pd.DataFrame):
                    print(f"Warning: The data for Mouse ID '{mouse_id}' in session '{session_number}' and block '{session_block}' is not a DataFrame. Skipping.")
                    continue

                # Copy the DataFrame and ensure the index is numeric
                df = item.copy()
                df.index = pd.to_numeric(df.index)

                # Process each column independently
                for column in df.columns:
                    event_time_data = df.loc[0:1, column]  # Data during the event (0 to +1 seconds)
                    post_event_data = df.loc[1:2, column]  # Data during the first second after the event (+1 to +2 seconds)

                    peak_response = event_time_data.max()  # Max response during the event
                    min_response = event_time_data.min()  # Minimum response during the event
                    mean_response_event = event_time_data.mean()  # Mean response during the event
                    mean_response_post_event = post_event_data.mean()  # Mean response during the post-event time
                    min_response_post_event = post_event_data.min()  #Minimum response during the post-event time
                    peak_response_post_event = post_event_data.max() #Maximum response during the post-event time

                    #Given Mean_moveX and Y being the row names in the movement df, the 1 second pre halt movement speeds are added
                    x_move = run_df.loc['Mean_moveX', column]
                    y_move = run_df.loc['Mean_moveY', column]
    
                    #add results to list of dicts
                    results.append({
                        "SessionNumber": session_number,
                        "SessionBlock": session_block,
                        "MouseID": mouse_id,
                        "EventTime": column,
                        "moveX": x_move,
                        "moveY": y_move,
                        "PeakResponse": peak_response,
                        "MinResponse":  min_response,
                        "MeanResponse": mean_response_event,
                        "MeanResponse_after": mean_response_post_event,
                        "MinResponse_after": min_response_post_event,
                        "PeakResponse_after": peak_response_post_event,
                    })

    # convert to a pandas df
    output_df = pd.DataFrame(results)
    return output_df



In [ ]:
extracted_variables = extract_aligned_vars(mouse_aligned, move_speeds)


In [ ]:
extracted_variables 

In [ ]:
extracted_variables_nohalt = extract_aligned_vars(mouse_aligned_nohalt, move_speeds_nohalt)


In [ ]:
extracted_variables_nohalt['event']='no halt'
extracted_variables['event']='halt'
combined_vars = pd.concat([extracted_variables_nohalt, extracted_variables])

combined_vars.to_csv('GRAB_MM_extracted_vars.csv', index=False)

In [ ]:
##Consider adding to the fitted model a continous variable which is the time column
#EventTime should currently be seconds from session start

# Look for correlation between running and fluorescence changes

In [ ]:
GRAB_MM_1

In [ ]:
subset = filter_data(GRAB_MM_1, ['B3M7', 'day1'])

In [ ]:
subset

In [ ]:
# Compute Pearson correlation
pearson_corr, pearson_pval = pearsonr(subset['movementX'], subset['470_dfF'])

# Compute Spearman correlation (handles nonlinear relationships better)
spearman_corr, spearman_pval = spearmanr(subset['movementX'], subset['470_dfF'])

print(f"Pearson correlation: {pearson_corr}, p-value: {pearson_pval}")
print(f"Spearman correlation: {spearman_corr}, p-value: {spearman_pval}")

In [ ]:
resampled = subset.sample(10000, random_state=9)  # Sample 10,000 rows
pearson_corr, pearson_pval = pearsonr(resampled['movementX'], resampled['470_dfF'])
spearman_corr, spearman_pval = spearmanr(resampled['movementX'], resampled['470_dfF'])
print(f"Subset Pearson correlation: {pearson_corr}, p-value: {pearson_pval}")
print(f"Subset Spearman correlation: {spearman_corr}, p-value: {spearman_pval}")

In [ ]:
import numpy as np
from scipy.stats import bootstrap

#data = np.array([GRAB_MM_1['movementX'], GRAB_MM_1['470_dfF']]).T
#ci = bootstrap((data,), np.corrcoef, confidence_level=0.95, n_resamples=1000)
#print(f"95% Confidence Interval for Pearson correlation: {ci.confidence_interval}")
